## install  and Import required packages





In [2]:
! pip install biopython
! pip install langchain
! pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.3/817.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.

In [3]:
from typing import Optional
from Bio import Entrez
from langchain.tools import BaseTool

In [4]:
# Import things that are needed generically
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from Bio import Entrez , Medline

## define custom tool to search pubmed and retrieve the abstract

In [ ]:
class PubMedSearch(BaseTool):
    name = "pubmed_search"
    description = "search PubMed for articles"

    def __init__(self, email: str):
      super().__init__()
      Entrez.email = email

    def _run(self, query: str) -> str:
        """
        Use the tool to search PubMed for articles and summarize their content.
        """
        # Set the email address to be used when making requests to PubMed
        handle = Entrez.esearch(db="pubmed", term=query, retmax=10)

        # Parse the search results using the read function
        search_results = Entrez.read(handle)

        # Get the list of PubMed IDs from the search results
        id_list = search_results["IdList"]

        # Use the efetch function to retrieve the documents with the given PubMed IDs
        handle = Entrez.efetch(db="pubmed", id=id_list, rettype="medline")
        records = list(Medline.parse(handle))

        # Loop through each record and print the abstract and content
        articles = []
        for record in records:
            articles.append(record.get("AB" , "?"))

        summary = "\n".join(articles)
        return {"Abstract": summary}

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        return self._run(query)



In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("COVID-19")




> Entering new AgentExecutor chain...
There is a lot of information available on COVID-19, so it's important to narrow down the question.
Action: pubmed_search
Action Input: "COVID-19 treatment"


Observation: Found 148931 articles on PubMed for query 'COVID-19 treatment'.
Thought:

That's a lot of articles, I should narrow it down further.
Action: pubmed_search
Action Input: "COVID-19 treatment clinical trials"


Observation: Found 10930 articles on PubMed for query 'COVID-19 treatment clinical trials'.
Thought:

That's still a lot of articles, I should focus on recent ones.
Action: pubmed_search
Action Input: "COVID-19 treatment clinical trials 2021"

...


Observation: Found 335 articles on PubMed for query 'COVID-19 monoclonal antibody treatment clinical trials 2021'.
Thought:

I should look for any promising results from these clinical trials.
Action: pubmed_search
Action Input: "COVID-19 monoclonal antibody treatment clinical trials 2021 results"


Observation: Found 175 articles on PubMed for query 'COVID-19 monoclonal antibody treatment clinical trials 2021 results'.
Thought:

Based on the articles I found, it seems that monoclonal antibody treatments for COVID-19 are showing promise in clinical trials. However, more research is needed to fully understand their effectiveness and safety.
Final Answer: Monoclonal antibody treatments for COVID-19 are showing promise in clinical trials, but more research is needed.

> Finished chain.


'Monoclonal antibody treatments for COVID-19 are showing promise in clinical trials, but more research is needed.'

## investigate BioPython package 

In [ ]:
from Bio import Entrez , Medline


In [ ]:
# Set the email address to be used when making requests to PubMed
Entrez.email = "your.email@example.com"

# Define the search term
search_term = "COVID-19"

# Use the esearch function to search PubMed for the search term
handle = Entrez.esearch(db="pubmed", term=search_term, retmax=10)

# Parse the search results using the read function
search_results = Entrez.read(handle)

# Get the list of PubMed IDs from the search results
id_list = search_results["IdList"]

# Use the efetch function to retrieve the documents with the given PubMed IDs
handle = Entrez.efetch(db="pubmed", id=id_list, rettype="xml", retmode="text")


In [ ]:
handle = Entrez.efetch(db="pubmed", id=id_list, rettype="medline")
records = Medline.parse(handle)
records

<generator object parse at 0x7fa904870740>

In [ ]:
records = list(records)
records

[{'PMID': '37163327',
  'OWN': 'NLM',
  'STAT': 'Publisher',
  'LR': '20230510',
  'IS': '1929-0748 (Print) 1929-0748 (Linking)',
  'VI': '12',
  'DP': '2023 May 10',
  'TI': 'Evaluating the Effect of Activity and Environment on Fall Risk in a Paradigm-Depending Laboratory Setting: Protocol for an Experimental Pilot Study.',
  'PG': 'e46930',
  'LID': '10.2196/46930 [doi]',
  'AB': 'BACKGROUND: Knowledge about the causal factors leading to falls is still limited, and fall prevention interventions urgently need to be more effective to limit the otherwise increasing burden caused by falls in older people. To identify individual fall risk, it is important to understand the complex interplay of fall-related factors. Although fall events are common, they are seldom observed, and fall reports are often biased. Due to the rapid development of wearable inertial sensors, an objective approach to capture fall events and the corresponding circumstances is provided. OBJECTIVE: The aim of this work

In [ ]:

for record in records:
    print("title:", record.get("TI", "?"))
    print("authors:", record.get("AU", "?"))
    print("source:", record.get("SO", "?"))
    print("############################")

title: Evaluating the Effect of Activity and Environment on Fall Risk in a Paradigm-Depending Laboratory Setting: Protocol for an Experimental Pilot Study.
authors: ['Sczuka KS', 'Schneider M', 'Schellenbach M', 'Kerse N', 'Becker C', 'Klenk J']
source: JMIR Res Protoc. 2023 May 10;12:e46930. doi: 10.2196/46930.
############################
title: Serosurveillance for Severe Acute Respiratory Syndrome Coronavirus 2 Antibody in Feral Swine and White-Tailed Deer in Texas.
authors: ['Palermo PM', 'Orbegozo J', 'Watts DM', 'Morrill JC']
source: Vector Borne Zoonotic Dis. 2023 May 10. doi: 10.1089/vbz.2023.0006.
############################
title: Investigating Social Media to Evaluate Emergency Medicine Physicians' Emotional Well-being During COVID-19.
authors: ['Agarwal AK', 'Mittal J', 'Tran A', 'Merchant R', 'Guntuku SC']
source: JAMA Netw Open. 2023 May 1;6(5):e2312708. doi: 10.1001/jamanetworkopen.2023.12708.
############################
title: Current trends in RNA virus detection th

In [ ]:
# Loop through each record and print the abstract and content
for record in records:
    print("Title:", record.get("TI", "?"))
    print("Abstract:", record.get("AB" , "?"))
    print("Content:", record.get("FAU" , "?")) 
    print("")
    print("-"*100)

Title: Evaluating the Effect of Activity and Environment on Fall Risk in a Paradigm-Depending Laboratory Setting: Protocol for an Experimental Pilot Study.
Abstract: BACKGROUND: Knowledge about the causal factors leading to falls is still limited, and fall prevention interventions urgently need to be more effective to limit the otherwise increasing burden caused by falls in older people. To identify individual fall risk, it is important to understand the complex interplay of fall-related factors. Although fall events are common, they are seldom observed, and fall reports are often biased. Due to the rapid development of wearable inertial sensors, an objective approach to capture fall events and the corresponding circumstances is provided. OBJECTIVE: The aim of this work is to operationalize a prototypical dynamic fall risk model regarding 4 ecologically valid real-world scenarios (opening a door, slipping, tripping, and usage of public transportation). We hypothesize that individual fa

## Query Biomedical Journal Citations in the PubMed Database

In [ ]:
import json 
import urllib.request 
import  time 

In [ ]:
import urllib.request
import json
import time

class e_utility(BaseModel):

    base_url_esearch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?"
    base_url_efetch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?"
    max_retry = 5
    sleep_time = 0.2

    def search(self, query):
        url = self.base_url_esearch + "db=pubmed&term=" + query + "&retmode=json&retmax=10&usehistory=y"
        result = urllib.request.urlopen(url)
        text = result.read().decode("utf-8")
        json_text = json.loads(text)

        articles = []
        webenv = json_text["esearchresult"]["webenv"]
        print(json_text["esearchresult"]["idlist"])
        for uid in json_text["esearchresult"]["idlist"]:
            article = self.retrieve_article(uid, webenv)
            articles.append(article)
        

        return json.dumps(articles)

    def retrieve_article(self, uid, webenv):
        url = self.base_url_efetch + "db=pubmed&retmode=xml&id=" + uid + "&webenv=" + webenv

        retry = 0
        while True:
            try:
                result = urllib.request.urlopen(url)
                break
            except urllib.error.HTTPError as e:
                if e.code == 429 and retry < self.max_retry:
                    # Too Many Requests error, wait for an exponentially increasing amount of time
                    print(f"Too Many Requests, waiting for {self.sleep_time:.2f} seconds...")
                    time.sleep(self.sleep_time)
                    self.sleep_time *= 2
                    retry += 1
                else:
                    raise e

        xml_text = result.read().decode("utf-8")

        # Get title
        title = ''
        if '<ArticleTitle>' in xml_text and '</ArticleTitle>' in xml_text:
            start_tag = '<ArticleTitle>'
            end_tag = '</ArticleTitle>'
            title = xml_text[xml_text.index(start_tag)+len(start_tag):xml_text.index(end_tag)]

        # Get abstract
        abstract = ''
        if '<AbstractText>' in xml_text and '</AbstractText>' in xml_text:
            start_tag = '<AbstractText>'
            end_tag = '</AbstractText>'
            abstract = xml_text[xml_text.index(start_tag)+len(start_tag):xml_text.index(end_tag)]

        # Return article as dictionary
        article = {'uid': uid, 'title': title, 'abstract': abstract}
        return article


In [ ]:

e = e_utility()
json_str = e.search("COVID-19")
articles = json.loads(json_str)

for article in articles[:3]:
    print("Title: " + article['title'])
    print("Abstract: " + article['abstract'])
    print("-"*100)


['37172383', '37172362', '37172333', '37172326', '37172319', '37172309', '37172274', '37172190', '37172174', '37172159']
Too Many Requests, waiting for 0.20 seconds...
Too Many Requests, waiting for 0.40 seconds...
Too Many Requests, waiting for 0.80 seconds...
Too Many Requests, waiting for 1.60 seconds...
Title: Antibiotic resistomes in face-mask biofilm along an urban river: Multiple drivers and co-occurrence with human opportunistic pathogens.
Abstract: Discarded face masks from the global COVID-19 pandemic have contributed significantly to plastic pollution in surface water, whereas their potential as a reservoir for aquatic pollutants is not well understood. Herein, we conducted a field experiment along a human-impacted urban river, investigating the variations of antibiotic resistance genes (ARGs), pathogens, and water-borne contaminants in commonly-used face masks. Results showed that high-biomass biofilms formed on face masks selectively enriched more ARGs than stone biofilm (

## Implemnt custom tool on langchin pubmed search  using Bio package

In [5]:
import logging
from typing import Any, Dict, List

from Bio import Entrez
from pydantic import BaseModel, Extra, root_validator
import requests
from io import StringIO , BytesIO

from langchain.schema import Document

logger = logging.getLogger(__name__)

In [71]:
class PubMedAPIWrapper(BaseModel):
    """
    Wrapper around PubMed API.

    This wrapper will use the PubMed API to conduct searches and fetch
    document summaries. By default, it will return the document summaries
    of the top-k results of an input search.

    Parameters:
        top_k_results: number of the top-scored document used for the PubMed tool
        load_max_docs: a limit to the number of loaded documents
        load_all_available_meta:
          if True: the `metadata` of the loaded Documents gets all available meta info
            (see https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ESearch)
          if False: the `metadata` gets only the most informative fields.
    """

    top_k_results: int = 3
    load_max_docs: int = 25
    ARXIV_MAX_QUERY_LENGTH = 300
    doc_content_chars_max: int = 2000
    load_all_available_meta: bool = False
    email :str="your_email@example.com"

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
    

    def run(self, query: str) -> str:
        """
        Run PubMed search and get the article meta information.
        See https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ESearch
        It uses only the most informative fields of article meta information.
        """

        try:
            docs = [
                f"Published: {result['pub_date']}\nTitle: {result['title']}\n"
                # f"Authors: {', '.join(author for author in result['authors'])}\n"
                f"Summary: {result['summary']}"
                for result in self.load(  # type: ignore
                    query[: self.ARXIV_MAX_QUERY_LENGTH]
                )
            ]
            return (
                "\n\n".join(docs)[: self.doc_content_chars_max]
                if docs
                else "No good Arxiv Result was found"
            )
        except :
            return "PubMed exceptio"
        

    def load(self, query: str):
        """
        Run PubMed search and get the article texts plus the article meta information.

        Returns: a list of documents with the document.page_content in text format

        """
        Entrez.email = self.email
        handle = Entrez.esearch(
            db="pubmed", term=query, retmax=self.load_max_docs
        )
        record = Entrez.read(handle)
        ids = record["IdList"]
        if len(ids) == 0:
            return []
        fetch_handle = Entrez.efetch(
            db="pubmed",
            id=ids,
            rettype="medline",
            retmode="text",
            retmax=self.load_max_docs,
        )
        
        docs = []
        records = list(Medline.parse(fetch_handle))
        for record in records:
            doc = {
                "summary": record.get("AB", ""),
                "title": record.get("TI", ""),
                "journal": record.get("JT", ""),
                "pub_date": record.get("DP", ""),
                "authors": record.get("AU", ""),
                "pmid": record.get("PMID", ""),
                "doi": record.get("DOI", ""),
            }
            docs.append(doc)

        return docs



In [72]:
"""Tool for the Pubmed API."""

from typing import Optional

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool


class PubmedQueryRun(BaseTool):
    """Tool that adds the capability to search using the PubMed API."""

    name = "PubMed"
    description = (
        "A wrapper around PubMed.org "
        "Useful for when you need to answer questions about Physics, Mathematics, "
        "Computer Science, Quantitative Biology, Quantitative Finance, Statistics, "
        "Electrical Engineering, and Economics "
        "from scientific articles on PubMed.org. "
        "Input should be a search query."
    )
    api_wrapper: PubMedAPIWrapper

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Arxiv tool."""
        return self.api_wrapper.run(query)

    async def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the PubMed tool asynchronously."""
        raise NotImplementedError("PubMedAPIWrapper does not support async")

In [73]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.0 , model_name="gpt-3.5-turbo" ,openai_api_key="your-openai-key")
api_wrapper = PubMedAPIWrapper()  # create an instance of PubMedAPIWrapper

tools = [PubmedQueryRun(api_wrapper=api_wrapper)]  ##  create an object of PubmedQueryRun with the api_wrapper instance

agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [74]:
## COVID-19
agent.run("What's the  COVID-19 ?")



> Entering new AgentExecutor chain...
I need to search for information on COVID-19 to answer this question.
Action: PubMed
Action Input: "COVID-19"


Observation: Published: 2023 May 8
Title: [Changes in Secondary Inorganic Ions in PM(2.5) at Different Pollution Stages Before and After COVID-19 Control].
Summary: To investigate the change characteristics of secondary inorganic ions in PM(2.5) at different pollution stages before and after COVID-19, the online monitoring of winter meteorological and atmospheric pollutant concentrations in Zhengzhou from December 15, 2019 to February 15, 2020 was conducted using a high-resolution (1 h) online instrument. This study analyzed the causes of the haze process of COVID-19, the diurnal variation characteristics of air pollutants, and the distribution characteristics of air pollutants at different stages of haze.The results showed that Zhengzhou was mainly controlled by the high-pressure ridge during the haze process, and the weather situation was stable, which was conducive to the accumulation of air pollutants. SNA was the main component of water-soluble ions, accounting for more than 90%.

This article is not relevant to the question asked. I need to refine my search query.
Action: PubMed
Action Input: "COVID-19 overview"


Observation: Published: 2023 Apr 29
Title: The Effects of Extrinsic and Intrinsic Factors on Neurogenesis.
Summary: In the mammalian brain, neurogenesis is maintained throughout adulthood primarily in two typical niches, the subgranular zone (SGZ) of the dentate gyrus and the subventricular zone (SVZ) of the lateral ventricles and in other nonclassic neurogenic areas (e.g., the amygdala and striatum). During prenatal and early postnatal development, neural stem cells (NSCs) differentiate into neurons and migrate to appropriate areas such as the olfactory bulb where they integrate into existing neural networks; these phenomena constitute the multistep process of neurogenesis. Alterations in any of these processes impair neurogenesis and may even lead to brain dysfunction, including cognitive impairment and neurodegeneration. Here, we first summarize the main properties of mammalian neurogenic niches to describe the cellular and molecular mechanisms of neurogenesis. Accumulating evidenc

This article is also not relevant to the question asked. I need to refine my search query further.
Action: PubMed
Action Input: "COVID-19 overview virus"


Observation: Published: 2023
Title: mRNA vaccines: a new opportunity for malaria, tuberculosis and HIV.
Summary: The success of the first licensed mRNA-based vaccines against COVID-19 has created a widespread interest on mRNA technology for vaccinology. As expected, the number of mRNA vaccines in preclinical and clinical development increased exponentially since 2020, including numerous improvements in mRNA formulation design, delivery methods and manufacturing processes. However, the technology faces challenges such as the cost of raw materials, the lack of standardization, and delivery optimization. MRNA technology may provide a solution to some of the emerging infectious diseases as well as the deadliest hard-to-treat infectious diseases malaria, tuberculosis, and human immunodeficiency virus/acquired immunodeficiency syndrome (HIV/AIDS), for which an effective vaccine, easily deployable to endemic areas is urgently needed. In this review, we discuss the functional structure, desig

This article is relevant to the question asked. 
Thought: I have found the information I need to answer the question. 
Final Answer: COVID-19 is a virus that causes a respiratory illness that can range from mild to severe, and can be fatal. It was first identified in Wuhan, China in December 2019 and has since spread globally, leading to a pandemic. Symptoms include fever, cough, and difficulty breathing, and it is primarily spread through respiratory droplets from an infected person. Vaccines have been developed and authorized for emergency use to help prevent the spread of the virus.

> Finished chain.


'COVID-19 is a virus that causes a respiratory illness that can range from mild to severe, and can be fatal. It was first identified in Wuhan, China in December 2019 and has since spread globally, leading to a pandemic. Symptoms include fever, cough, and difficulty breathing, and it is primarily spread through respiratory droplets from an infected person. Vaccines have been developed and authorized for emergency use to help prevent the spread of the virus.'

## Implemnt custom tool on langchin pubmed search  without Bio package

In [59]:
import json
import time
import urllib.error
import urllib.request
from typing import List
from pydantic import BaseModel, Extra


class PubMedAPIWrapper(BaseModel):
    """
    Wrapper around PubMed API.

    This wrapper will use the PubMed API to conduct searches and fetch
    document summaries. By default, it will return the document summaries
    of the top-k results of an input search.

    Parameters:
        top_k_results: number of the top-scored document used for the PubMed tool
        load_max_docs: a limit to the number of loaded documents
        load_all_available_meta:
          if True: the `metadata` of the loaded Documents gets all available meta info
            (see https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ESearch)
          if False: the `metadata` gets only the most informative fields.
    """

    base_url_esearch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?"
    base_url_efetch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?"
    max_retry = 5
    sleep_time = 0.2

    # Default values for the parameters
    top_k_results: int = 3
    load_max_docs: int = 25
    ARXIV_MAX_QUERY_LENGTH = 300
    doc_content_chars_max: int = 2000
    load_all_available_meta: bool = False
    email :str="your_email@example.com"

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
    
    def run(self, query: str) -> str:
        """
        Run PubMed search and get the article meta information.
        See https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ESearch
        It uses only the most informative fields of article meta information.
        """

        # try:
        # Retrieve the top-k results for the query
        docs = [
            f"Published: {result['pub_date']}\nTitle: {result['title']}\n"
            f"Summary: {result['summary']}"
            for result in self.search(query[:self.ARXIV_MAX_QUERY_LENGTH])
        ]

        # Join the results and limit the character count
        return ("\n\n".join(docs)[:self.doc_content_chars_max]
                if docs
                else "No good PubMed Result was found")
        # except:
        #     return "Arxiv exception"
    
    def search(self, query: str) -> List[dict]:
        """
        Search PubMed for documents matching the query.
        Return a list of dictionaries containing the document metadata.
        """

        url = (self.base_url_esearch + "db=pubmed&term=" + str({urllib.parse.quote(query)}) +
               f"&retmode=json&retmax={self.top_k_results}&usehistory=y")
        result = urllib.request.urlopen(url)
        text = result.read().decode("utf-8")
        json_text = json.loads(text)

        articles = []
        webenv = json_text["esearchresult"]["webenv"]
        print(json_text["esearchresult"]["idlist"])
        for uid in json_text["esearchresult"]["idlist"]:
            article = self.retrieve_article(uid, webenv)
            articles.append(article)
        
        # Convert the list of articles to a JSON string
        return articles 

    def retrieve_article(self, uid, webenv):
        url = self.base_url_efetch + "db=pubmed&retmode=xml&id=" + uid + "&webenv=" + webenv

        retry = 0
        while True:
            try:
                result = urllib.request.urlopen(url)
                break
            except urllib.error.HTTPError as e:
                if e.code == 429 and retry < self.max_retry:
                    # Too Many Requests error, wait for an exponentially increasing amount of time
                    print(f"Too Many Requests, waiting for {self.sleep_time:.2f} seconds...")
                    time.sleep(self.sleep_time)
                    self.sleep_time *= 2
                    retry += 1
                else:
                    raise e

        xml_text = result.read().decode("utf-8")

        # Get title
        title = ''
        if '<ArticleTitle>' in xml_text and '</ArticleTitle>' in xml_text:
            start_tag = '<ArticleTitle>'
            end_tag = '</ArticleTitle>'
            title = xml_text[xml_text.index(start_tag)+len(start_tag):xml_text.index(end_tag)]

        # Get abstract
        abstract = ''
        if '<AbstractText>' in xml_text and '</AbstractText>' in xml_text:
            start_tag = '<AbstractText>'
            end_tag = '</AbstractText>'
            abstract = xml_text[xml_text.index(start_tag)+len(start_tag):xml_text.index(end_tag)]

        # Get publication date
        pub_date = ''
        if '<PubDate>' in xml_text and '</PubDate>' in xml_text:
            start_tag = '<PubDate>'
            end_tag = '</PubDate>'
            pub_date = xml_text[xml_text.index(start_tag)+len(start_tag):xml_text.index(end_tag)]

        # Return article as dictionary
        article = {'uid': uid, 'title': title, 'summary': abstract, 'pub_date': pub_date}
        return article



In [60]:
util=PubMedAPIWrapper()
result=util.run("covid-19")

['37177918', '37177879', '37177872']


In [61]:
result

'Published: <Year>2023</Year><Month>May</Month><Day>08</Day>\nTitle: [Changes in Secondary Inorganic Ions in PM<sub>2.5</sub> at Different Pollution Stages Before and After COVID-19 Control].\nSummary: To investigate the change characteristics of secondary inorganic ions in PM<sub>2.5</sub> at different pollution stages before and after COVID-19, the online monitoring of winter meteorological and atmospheric pollutant concentrations in Zhengzhou from December 15, 2019 to February 15, 2020 was conducted using a high-resolution (1 h) online instrument. This study analyzed the causes of the haze process of COVID-19, the diurnal variation characteristics of air pollutants, and the distribution characteristics of air pollutants at different stages of haze.The results showed that Zhengzhou was mainly controlled by the high-pressure ridge during the haze process, and the weather situation was stable, which was conducive to the accumulation of air pollutants. SNA was the main component of wate

In [62]:
"""Tool for the Pubmed API."""

from typing import Optional

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool


class PubmedQueryRun(BaseTool):
    """Tool that adds the capability to search using the PubMed API."""

    name = "PubMed"
    description = (
        "A wrapper around PubMed.org "
        "Useful for when you need to answer questions about Physics, Mathematics, "
        "Computer Science, Quantitative Biology, Quantitative Finance, Statistics, "
        "Electrical Engineering, and Economics "
        "from scientific articles on PubMed.org. "
        "Input should be a search query."
    )
    api_wrapper: PubMedAPIWrapper

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Arxiv tool."""
        return self.api_wrapper.run(query)

    async def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the PubMed tool asynchronously."""
        raise NotImplementedError("PubMedAPIWrapper does not support async")

In [65]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.0 ,model_name="gpt-3.5-turbo" , openai_api_key="your-openai-key")
api_wrapper = PubMedAPIWrapper()  # create an instance of PubMedAPIWrapper

tools = [PubmedQueryRun(api_wrapper=api_wrapper)]  ##  create an object of PubmedQueryRun with the api_wrapper instance

agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [66]:
## COVID-19
agent.run("What's the  COVID-19 ?")



> Entering new AgentExecutor chain...
I need to search for information on COVID-19 to answer this question.
Action: PubMed
Action Input: "COVID-19"['37177918', '37177879', '37177872']

Observation: Published: <Year>2023</Year><Month>May</Month><Day>08</Day>
Title: [Changes in Secondary Inorganic Ions in PM<sub>2.5</sub> at Different Pollution Stages Before and After COVID-19 Control].
Summary: To investigate the change characteristics of secondary inorganic ions in PM<sub>2.5</sub> at different pollution stages before and after COVID-19, the online monitoring of winter meteorological and atmospheric pollutant concentrations in Zhengzhou from December 15, 2019 to February 15, 2020 was conducted using a high-resolution (1 h) online instrument. This study analyzed the causes of the haze process of COVID-19, the diurnal variation characteristics of air pollutants, and the distribution characteristics of air pollutants at different stages of haze.The results showed that Zhengzhou was main


Observation: Published: <Year>2023</Year>
Title: mRNA vaccines: a new opportunity for malaria, tuberculosis and HIV.
Summary: The success of the first licensed mRNA-based vaccines against COVID-19 has created a widespread interest on mRNA technology for vaccinology. As expected, the number of mRNA vaccines in preclinical and clinical development increased exponentially since 2020, including numerous improvements in mRNA formulation design, delivery methods and manufacturing processes. However, the technology faces challenges such as the cost of raw materials, the lack of standardization, and delivery optimization. MRNA technology may provide a solution to some of the emerging infectious diseases as well as the deadliest hard-to-treat infectious diseases malaria, tuberculosis, and human immunodeficiency virus/acquired immunodeficiency syndrome (HIV/AIDS), for which an effective vaccine, easily deployable to endemic areas is urgently needed. In this review, we discuss the functional str

I have found a relevant article on COVID-19. 
Final Answer: COVID-19 is a virus that causes a respiratory illness and has potential secondary complications, including cardiovascular sequelae. It has led to a global effort to understand and manage its impact.

> Finished chain.


'COVID-19 is a virus that causes a respiratory illness and has potential secondary complications, including cardiovascular sequelae. It has led to a global effort to understand and manage its impact.'